In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
import pandas as pd
import numpy as np
import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2
import os
import time
from google.colab import drive
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# Google Drive 마운트
drive.mount('/content/drive', force_remount=True)

class DiecastingDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.transform:
            transformed = self.transform(image=image)
            image = transformed['image']

        label = self.labels[idx]
        return image, label

def get_data_paths(data_base_path):
    class_dirs = {
        'BLDC-400-Back': 0,
        'BLDC-400-Front': 1,
        'G2': 2,
        'CN7': 3
    }
    paths = []
    labels = []

    for class_dir, label in class_dirs.items():
        class_path = os.path.join(data_base_path, class_dir)
        image_files = [os.path.join(class_path, f) for f in os.listdir(class_path)
                      if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
        paths.extend(image_files)
        labels.extend([label] * len(image_files))

    return paths, labels

# 데이터 전처리 정의 (resize만 사용)
transforms = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

def train_model(model, train_loader, val_loader, device, epochs, lr):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    train_start_time = time.time()
    best_val_acc = 0.0

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        train_acc = 100. * correct / total
        train_loss = running_loss / len(train_loader)
        print(f'Epoch [{epoch+1}/{epochs}], Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')

        # 검증
        model.eval()
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for val_images, val_labels in val_loader:
                val_images, val_labels = val_images.to(device), val_labels.to(device)
                val_outputs = model(val_images)
                _, val_predicted = val_outputs.max(1)
                val_total += val_labels.size(0)
                val_correct += val_predicted.eq(val_labels).sum().item()

        val_acc = 100. * val_correct / val_total
        print(f'Epoch [{epoch+1}/{epochs}], Val Acc: {val_acc:.2f}%')

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), f'/content/drive/MyDrive/{model.__class__.__name__}_best.pth')

    training_time = time.time() - train_start_time
    return best_val_acc, training_time

def evaluate_model(model, test_loader, device, class_names):
    model.eval()
    all_preds = []
    all_labels = []
    inference_times = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            # 추론 시간 측정
            start_time = time.time()
            outputs = model(images)
            inference_times.append(time.time() - start_time)

            _, predicted = outputs.max(1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # 성능 지표 계산
    accuracy = 100. * sum(np.array(all_preds) == np.array(all_labels)) / len(all_labels)
    avg_inference_time = np.mean(inference_times)
    cm = confusion_matrix(all_labels, all_preds)
    report = classification_report(all_labels, all_preds, target_names=class_names, output_dict=True)

    return accuracy, avg_inference_time, cm, report

def main():
    # 장치 설정
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f'Using device: {device}')

    # 데이터 경로 설정
    train_base_path = '/content/drive/MyDrive/data_augmented'
    test_base_path = '/content/drive/MyDrive/test_data'

    # 데이터 로드
    train_paths, train_labels = get_data_paths(train_base_path)
    test_paths, test_labels = get_data_paths(test_base_path)

    # 데이터셋 생성
    train_dataset = DiecastingDataset(train_paths, train_labels, transforms)
    test_dataset = DiecastingDataset(test_paths, test_labels, transforms)

    # 데이터 로더 생성
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    # 클래스 이름 정의
    class_names = ['BLDC-400-Back', 'BLDC-400-Front', 'G2', 'CN7']

    # 모델 정의
    models_to_test = {
        'EfficientNet-B4': models.efficientnet_b4(weights='DEFAULT'),
        'EfficientNet-B5': models.efficientnet_b5(weights='DEFAULT'),
        'EfficientNet-V2-S': models.efficientnet_v2_s(weights='DEFAULT')
    }

    # 결과 저장용 리스트
    results = []

    # 각 모델 평가
    for model_name, model in models_to_test.items():
        print(f'\nTraining and evaluating {model_name}...')

        # 분류층 수정
        if 'v2' in model_name.lower():
            model.classifier = nn.Linear(model.classifier[1].in_features, 4)
        else:
            model.classifier[1] = nn.Linear(model.classifier[1].in_features, 4)

        model = model.to(device)

        # 학습
        best_val_acc, training_time = train_model(
            model, train_loader, test_loader, device,
            epochs=2, lr=1e-4
        )

        # 테스트셋 평가
        accuracy, inf_time, cm, report = evaluate_model(
            model, test_loader, device, class_names
        )

        # 결과 저장
        results.append({
            'Model': model_name,
            'Accuracy': accuracy,
            'Training Time': training_time,
            'Inference Time': inf_time,
            'Precision': report['weighted avg']['precision'],
            'Recall': report['weighted avg']['recall'],
            'F1-Score': report['weighted avg']['f1-score']
        })

        # 혼동 행렬 저장
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                   xticklabels=class_names, yticklabels=class_names)
        plt.title(f'Confusion Matrix - {model_name}')
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        plt.savefig(f'/content/drive/MyDrive/{model_name}_confusion_matrix_.png')
        plt.close()

    # 결과 DataFrame 생성 및 저장
    results_df = pd.DataFrame(results)
    results_df.to_csv('/content/drive/MyDrive/model_comparison_results_.csv', index=False)

    # 결과 시각화
    plt.figure(figsize=(15, 5))

    # 정확도 비교
    plt.subplot(1, 3, 1)
    plt.bar(results_df['Model'], results_df['Accuracy'])
    plt.title('Model Accuracy Comparison')
    plt.xticks(rotation=45)
    plt.ylabel('Accuracy (%)')

    # 학습 시간 비교
    plt.subplot(1, 3, 2)
    plt.bar(results_df['Model'], results_df['Training Time'])
    plt.title('Training Time Comparison')
    plt.xticks(rotation=45)
    plt.ylabel('Time (seconds)')

    # F1-Score 비교
    plt.subplot(1, 3, 3)
    plt.bar(results_df['Model'], results_df['F1-Score'])
    plt.title('F1-Score Comparison')
    plt.xticks(rotation=45)
    plt.ylabel('F1-Score')

    plt.tight_layout()
    plt.savefig('/content/drive/MyDrive/model_comparison_plots_.png')
    plt.close()

    print("\nResults have been saved to 'model_comparison_results_.csv'")
    print("Plots have been saved as 'model_comparison_plots.png'")

if __name__ == '__main__':
    main()

Mounted at /content/drive
Using device: cuda

Training and evaluating EfficientNet-B4...
Epoch [1/2], Train Loss: 1.2588, Train Acc: 68.63%
Epoch [1/2], Val Acc: 94.12%
Epoch [2/2], Train Loss: 0.9637, Train Acc: 98.53%
Epoch [2/2], Val Acc: 99.75%

Training and evaluating EfficientNet-B5...
Epoch [1/2], Train Loss: 0.8473, Train Acc: 86.03%
Epoch [1/2], Val Acc: 99.51%
Epoch [2/2], Train Loss: 0.1935, Train Acc: 100.00%
Epoch [2/2], Val Acc: 100.00%

Training and evaluating EfficientNet-V2-S...
Epoch [1/2], Train Loss: 0.6029, Train Acc: 89.71%
Epoch [1/2], Val Acc: 100.00%
Epoch [2/2], Train Loss: 0.0517, Train Acc: 100.00%
Epoch [2/2], Val Acc: 100.00%

Results have been saved to 'model_comparison_results_.csv'
Plots have been saved as 'model_comparison_plots.png'
